In [8]:
import os
print("TESSDATA_PREFIX:", os.environ.get("TESSDATA_PREFIX"))
os.environ["TESSDATA_PREFIX"] = "/home/fangj1/miniconda3/envs/RAG/share/tessdata/"

! tesseract image.png output.txt -l eng

TESSDATA_PREFIX: /home/fangj1/miniconda3/envs/RAG/share/
Estimating resolution as 283


In [15]:
def ocr_extract_structured_data(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    extracted_text = pytesseract.image_to_string(binary, lang='eng')
    
    print("Extracted Text:\n", extracted_text)
    lines = [line.strip() for line in extracted_text.splitlines() if line.strip()]
    data = {}
    current_key = None
    
    for line in lines:
        if ':' in line:
            key, value = line.split(':', 1)
            current_key = key.strip()
            data[current_key] = value.strip()
        elif current_key:
            data[current_key] += f" {line.strip()}"
    
    return json.dumps(data, indent=4, ensure_ascii=False)

image_path = "./image.png"  
structured_data = ocr_extract_structured_data(image_path)
print("Structured Data in JSON Format:\n", structured_data)

Extracted Text:
 To: Amazon.com From: Seagull Scientific
6001 Bethlehem Blvd 15325 SE 30th Place
Baltimore, MD 21219 Bellevue, WA 98007

UPC: 123456789

Carrier: UPS Pro# 090711935
Bill of Lading # 0001258108

1 of 6 pallets


Structured Data in JSON Format:
 {
    "To": "Amazon.com From: Seagull Scientific 6001 Bethlehem Blvd 15325 SE 30th Place Baltimore, MD 21219 Bellevue, WA 98007",
    "UPC": "123456789",
    "Carrier": "UPS Pro# 090711935 Bill of Lading # 0001258108 1 of 6 pallets"
}
